# 3. Implementing Google Maps API

In [1]:
# Import the dependencies used in the project
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import google_key

# # Format numbers in DataFrames to have two decimal places
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Load the data created in part 1
city_df = pd.read_csv("Data/City Data.csv")

# Convert Date to a readable format
city_df["Date"] = pd.to_datetime(city_df["Date"],unit="s")
city_df["Date"] = city_df["Date"].dt.date

### 3.1 Humidity Heatmap

In [2]:
# Configure gmaps and call location and humidity data to plot on the map
gmaps.configure(api_key = google_key)
coords = city_df[["Lat", "Lng"]]
humidity = city_df["Humidity"]
max_humidity = humidity.max()

In [3]:
# Create the map
fig = gmaps.figure()

# Create a new layer for the heatmap
heat_layer = gmaps.heatmap_layer(coords,
                                 weights = humidity,
                                 max_intensity = max_humidity,
                                 point_radius = 6)

# Add the newly created heatmap layer and display the final map
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### 3.2 Finding Hotels Using Weather Criteria
For this example, we will look at cities with temperatures above 70F, below 80F and wind speed below 10mph.

In [4]:
# Create a new DataFrame for the above criteria
filtered_cities = city_df.loc[(city_df["Max Temp"] > 70)
                              & (city_df["Max Temp"] < 80)
                              & (city_df["Wind Speed"] < 10)]

# Shuffle the new cities so a random sample can be taken later on
filtered_cities = filtered_cities.sample(frac = 1)
filtered_cities.reset_index(inplace = True)
filtered_cities

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,381,phalombe,-15.81,35.65,76.23,38,63,2.35,MW,2022-09-25
1,30,argelia,2.26,-77.25,78.62,65,96,2.33,CO,2022-09-25
2,343,nola,40.92,14.53,70.38,99,100,3.80,IT,2022-09-25
3,233,kampot,10.62,104.18,77.05,87,100,1.86,KH,2022-09-25
4,347,nova venecia,-18.71,-40.40,77.99,57,94,9.44,BR,2022-09-25
...,...,...,...,...,...,...,...,...,...,...
100,338,nepa nagar,21.47,76.40,75.72,77,57,2.48,IN,2022-09-25
101,261,kriel,-26.27,29.23,71.78,34,0,7.63,ZA,2022-09-25
102,267,labuhan,-6.88,112.21,78.13,82,98,8.61,ID,2022-09-25
103,163,gaoual,11.75,-13.20,79.50,91,84,2.95,GN,2022-09-25


There are a lot of hotels matching the criteria, but we only want to have 10 markers in total on the final map.

In [6]:
# Create a copy of the filterest cities DataFrame
hotels = filtered_cities

# Track the number of hotels logged so no more than 10 are in the final map
hotelcount = 0

# Set the parameters for a google search
params = {
    "radius": 5000,
    "types": "lodging",
    "key": google_key}

# Create a loop to find the hotel information
for index, row in hotels.iterrows():
    # End loop if hotel quota is met
    if hotelcount == 10:
        break
    
    # Create the location information
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make a search through the API
    response = requests.get(base_url, params = params).json()
    results = response["results"]
    
    # Add the hotel information
    try:
        hotels.loc[index, "Hotel Name"] = results[0]["name"]
        hotelcount = hotelcount + 1
    
    # Skip if no hotel found
    except (keyError, IndexError):
        hotels.loc[index, "Hotel Name"] = None

In [8]:
# Create a new DataFrame to get the marker data
markers = hotels.dropna()
markers

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,381,phalombe,-15.81,35.65,76.23,38,63,2.35,MW,2022-09-25,RIVONIA
1,30,argelia,2.26,-77.25,78.62,65,96,2.33,CO,2022-09-25,Hotel San Fernando Argelia
2,343,nola,40.92,14.53,70.38,99,100,3.80,IT,2022-09-25,Palazzo Giordano Bruno
3,233,kampot,10.62,104.18,77.05,87,100,1.86,KH,2022-09-25,Two Moons Hotel
4,347,nova venecia,-18.71,-40.40,77.99,57,94,9.44,BR,2022-09-25,Hotel Ferrari - Rede Ferrari
5,470,show low,34.25,-110.03,74.08,33,0,5.75,US,2022-09-25,Hampton Inn & Suites Show Low-Pinetop
6,256,korla,41.76,86.15,72.46,30,33,8.55,CN,2022-09-25,Gulisitan Hotel
7,48,barcelos,41.54,-8.62,72.68,44,27,7.76,PT,2022-09-25,Hotel Do Terço
8,248,kitsuki,33.42,131.62,70.20,85,53,0.78,JP,2022-09-25,Hotel Silk Road
9,514,tura,25.52,90.22,72.43,96,100,3.44,IN,2022-09-25,Hotel RIKMAN Continental


In [10]:
# Add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in markers.iterrows()]
locations = markers[["Lat", "Lng"]]

In [11]:
# Add marker layer to the map and display
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))